# An Interactive Monty Hall Simulation

`nbinteract` was designed to make interactive explanations easy to create.

In this section, you will create an interactive simulation of the Monty Hall Problem. You may continue writing code in the notebook from the previous section or create a new one for this section.

## The Monty Hall Problem

The Monty Hall Problem ([Wikipedia][wiki]) is a famous probability problem that has stumped many, mathematicians included. The problem goes something like this:

> Suppose you're on a game show, and you're given the choice of three doors: Behind one door is a car; behind the others, goats. You pick a door, say No. 1, and the host, who knows what's behind the doors, opens another door, say No. 3, which has a goat. He then says to you, "Do you want to switch your choice to door No. 2?" Is it to your advantage to switch your choice?

Perhaps unintuitively, you will win the prize about twice as often if you switch doors. We can show this through simulation.

[wiki]: https://en.wikipedia.org/wiki/Monty_Hall_problem